# State Task Networks

This notebook implements in Pyomo the simple state task network described in Section 3 of Kondili, Pantelides & Sargent.

In [1]:
from pyomo.environ import *
import itertools

## Sets

The set $I_j$ is the set of tasks that can be performed by unit $j$.

In [2]:
map_I_J = {'J1': ['I1', 'I2'],
           'J2': ['I1']}

In [3]:
model = ConcreteModel()
model.J = Set(initialize=map_I_J.keys(), doc='Unit')
unique_I = set(itertools.chain.from_iterable(map_I_J.values()))
model.I = Set(initialize=unique_I, doc='Task')
model.I_J = Set(within=model.I * model.J, initialize=[(i, j) for j in map_I_J for i in map_I_J[j]],
                doc='Tasks that can be performed by unit j')
model.I_J.pprint()

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
I_J : Tasks that can be performed by unit j
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain     : Size : Members
    None :     2 : I_J_domain :    3 : {('I1', 'J1'), ('I2', 'J1'), ('I1', 'J2')}


In [4]:
model.T = Set(initialize=[0, 1, 2, 3], doc='Time Period')
model.States = Set(initialize=['S1', 'S2', 'S3'], doc='State')

Task $i$ is defined by the following dependent sets

In [5]:
map_S_I = {'I1': ['S1', 'S2'],
           'I2': ['S3']}
map_Sbar_I = {'I1': ['S1', 'S2'],
              'I2': ['S3']}

In [6]:
model.S_I = Set(within=model.States * model.I, initialize=[(s, i) for i in map_S_I for s in map_S_I[i]], 
                doc='States that feed task i')
model.Sbar_I = Set(within=model.States * model.I, initialize=[(s, i) for i in map_Sbar_I for s in map_Sbar_I[i]],
                   doc='States produced by task i')
model.S_I.pprint()

S_I : States that feed task i
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain     : Size : Members
    None :     2 : S_I_domain :    3 : {('S1', 'I1'), ('S2', 'I1'), ('S3', 'I2')}


## Parameters

In [7]:
model.M = Param(initialize=1000, doc='Allocation constraint parameter')
model.rho = Param(model.I, model.States, doc='Proportion of input of task i from state s in model.S')

## Variables

In [8]:
model.W = Var(model.I_J, model.T, doc='Start task k using unit j at time t', within=Binary)
model.W.pprint()

W : Start task k using unit j at time t
    Size=12, Index=W_index
    Key             : Lower : Value : Upper : Fixed : Stale : Domain
    ('I1', 'J1', 0) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J1', 1) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J1', 2) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J1', 3) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 0) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 1) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 2) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 3) :     0 :  None :     1 : False :  True : Binary
    ('I2', 'J1', 0) :     0 :  None :     1 : False :  True : Binary
    ('I2', 'J1', 1) :     0 :  None :     1 : False :  True : Binary
    ('I2', 'J1', 2) :     0 :  None :     1 : False :  True : Binary
    ('I2', 'J1', 3) :     0 :  None :     1 : False :  True : Binary


In [9]:
model.B = Var(model.I_J, model.T, doc='Amount of material which starts undergoing task k in unit j at beginning of t')
model.Stored = Var(model.States, model.T, doc='Amount of material stored in state s at beginning of t')

## Constraints

In [10]:
def allocation_constraints_rule(model, i, j, t):
    lhs = sum(model.W[idash, j, tdash] for tdash in model.T for idash in model.I_J[j] 
              if tdash >= t and tdash <= t + p[i] - 1) - 1
    rhs = model.M * (1 - model.W[i, j, t])
    return lhs <= rhs
model.allocation_constraints = Constraint(model.I_J, model.T, rule=allocation_constraints_rule)

ERROR: Rule failed when generating expression for constraint
    allocation_constraints with index ('I1', 'J1', 0): IndexError: I_J indices
    must be integers, not str
ERROR: Constructing component 'allocation_constraints' from data=None failed:
    IndexError: I_J indices must be integers, not str


IndexError: I_J indices must be integers, not str